In [ ]:
%run ../chap05/dataset.ipynb

In [1]:
from openpyxl import load_workbook

class transformer_Dataset(Dataset):
    def __init__(self, filenames, timesteps=10):
        super(transformer_Dataset, self).__init__('NLP', 'binary')
        self.training_target_corpus=[] #
        self.training_original_corpus=[]  #index
        self.original_counting_corpus = {} #
        self.counting_list=[] #
        self.target_word_to_id = {} #
        self.target_id_to_word = {} #
        self.original_word_to_id = {}  #
        self.original_id_to_word = {}  #
        self.training_original_one_hot_word = 0   #
        self.training_target_one_hot_word=0 #
        self.original_unique_words = 0 #
        self.negative=[]
        self.most_length=0      #
        self.target_sentence_to_id = {}
        self.target_id_to_sentence = {}  #
        self.original_sentence_to_id = {}  
        self.original_id_to_sentence = {}  #
        self.training_one_hot_sentence = 0
        self.original_unique_sentence = 0 #
        self.original_text=[]  #raw
        self.target_text=[]   #
        self.original_token_text=[]   #
        self.Max_data=9    #
        self.most_length_target=0
        
    def text_find_number_words(self,name, training):
        full_text=list()
        name='text_data/'+name
        load_wb = load_workbook(name, data_only=True)
        load_ws = load_wb['Sheet1']
        #print(load_ws.rows)
        for i, row in enumerate(load_ws.iter_rows(min_row=1, max_row=1309, min_col=1, max_col=20000, values_only=True)):
            #print(i)
            if i ==self.Max_data:
                break
            temp_list=list()
            for j, cell in enumerate(row):
                if j == 0:
                    self.original_text.append(str(cell).lower())
                    #print(cell.lower())
                    continue
                if cell !=None:
                    temp_list.append(str(cell).lower())
       
                    pass
                else:
                    for word in temp_list:
                        if word not in self.target_word_to_id:
                            new_id = len(self.target_word_to_id)
                            self.target_word_to_id[word] = new_id
                            self.target_id_to_word[new_id] = word
                    if self.most_length_target<len(temp_list):
                        self.most_length_target=len(temp_list)
                    
                    self.training_target_corpus.append([self.target_word_to_id[w] for w in temp_list])
                    self.target_id_to_sentence[i] = temp_list
                    self.target_text.append(' '.join(temp_list))
                  
                    break
        #self.most_length_target=1
        self.most_length_target+=1
        self.target_word_to_id['sep0'] = len(self.target_word_to_id)
        self.target_id_to_word[len(self.target_word_to_id)-1]='sep0'
        self.target_word_to_id['start0'] = len(self.target_word_to_id)
        self.target_id_to_word[len(self.target_word_to_id)-1]='start0'
        #print(self.target_id_to_word)
        for i in range(self.Max_data):
            self.training_target_corpus[i]=[len(self.target_word_to_id)-1]+self.training_target_corpus[i]
        #print(self.training_target_corpus)
        for i, row_origin in enumerate(self.original_text):
            words_origin = row_origin.lower().replace('.',' .').replace(',',' ,').split(' ')
            self.original_token_text.append(words_origin)
            if self.most_length<len(words_origin):
                self.most_length=len(words_origin)
            for word in words_origin:
                if word not in self.original_word_to_id:
                    new_id = len(self.original_word_to_id)
                    self.original_word_to_id[word] = new_id
                    self.original_id_to_word[new_id] = word
                
            self.training_original_corpus.append([self.original_word_to_id[w] for w in words_origin])
            self.original_id_to_sentence[i] = row_origin
        self.training_original_corpus_numpy=np.zeros((self.Max_data,self.most_length), dtype='int32')
        self.training_target_corpus_numpy=np.zeros((self.Max_data,self.most_length_target), dtype='int32')
        for i in range(self.Max_data):
            for j in range(self.most_length):
                try:
                    self.training_original_corpus_numpy[i,j]=self.training_original_corpus[i][j]
                except:
                    self.training_original_corpus_numpy[i,j]=len(self.original_word_to_id)
        for i in range(self.Max_data):
            for j in range(self.most_length_target):
                try:
                    self.training_target_corpus_numpy[i,j]=self.training_target_corpus[i][j]
                except:
                    self.training_target_corpus_numpy[i,j]=len(self.target_word_to_id)-2
        
        if training==1:
            self.training_original_one_hot_word = np.eye(len(self.original_word_to_id),dtype = 'float64')
            self.training_target_one_hot_word = np.eye(len(self.target_word_to_id),dtype = 'float64')
            self.original_unique_words = len(self.original_word_to_id)
            self.target_unique_words = len(self.target_word_to_id)
            #print("dd",self.target_word_to_id)
            self.original_unique_sentence = len(self.original_id_to_sentence)
            """
            sum_=0
            for i in range(self.original_unique_sentence):
                for j in self.training_original_corpus[i]:
                    if j not in self.original_counting_corpus:
                        self.original_counting_corpus[j] = 1
                        sum_+=1
                    else:
                        self.original_counting_corpus[j]+=1
                        sum_+=1
            for i in range(len(self.original_counting_corpus)):
                self.counting_list.append(self.original_counting_corpus[i]/sum_)
            """
        else:
            pass
        #self.training_original_corpus=np.array(self.training_original_corpus)
        #self.training_target_corpus=np.array(self.training_target_corpus)
        print("input most_length",self.most_length)
    @property
    def train_count(self):
        return len(self.training_original_corpus)
    
    def __str__(self):
        return '{}({}, {} train_data)'.format(self.name, self.mode, self.train_count)
    
    def get_train_data(self, batch_size, nth):
        from_idx = nth * batch_size
        to_idx = (nth + 1) * batch_size
    

        tr_X = self.training_original_corpus_numpy[self.indices[from_idx:to_idx]]
        tr_Y = self.training_target_corpus_numpy[self.indices[from_idx:to_idx]]
        
        return tr_X, tr_Y

    def shuffle_train_data(self, size):
        np.random.seed(int(time.time()))
        self.indices = np.arange(size)
        np.random.shuffle(self.indices)
        


      



NameError: name 'Dataset' is not defined